In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/Othercomputers/My Laptop/Documents/UCA DSAI/Internship 2/Code/text-models/scripts_and_notebooks/

/content/drive/Othercomputers/My Laptop/Documents/UCA DSAI/Internship 2/Code/text-models/scripts_and_notebooks


In [3]:
%cd ..

/content/drive/Othercomputers/My Laptop/Documents/UCA DSAI/Internship 2/Code/text-models


In [5]:
pip install lit_nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 746 kB 4.3 MB/s 
     |████████████████████████████████| 92 kB 10.8 MB/s 
     |████████████████████████████████| 88 kB 7.6 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 1.1 MB 47.3 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=8c675587ea3375a69d44e3d5d5eb42e939dd5e79eef97d634e6efdebd4c2fe3d
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
  Created wheel for umap-learn: filename=umap_learn-0.5.3-py3-none-any.whl size=82829 sha256=ec6785bd9f981e6fd0ce778d818ca03c1960d68315402722a2d623490bac63f7
  Stored in directory: /root/.cache/pip/wheels/b3/52/a5/1fd9e3e76a7ab34f134c07469cd6f16e27ef3a37aeff1fe821
  Created wheel for pynndescent: filename=pynndescent-0.5.7-py3-none-any.whl si

In [14]:
!python scripts_and_notebooks/lit/run.py

2022-07-24 17:18:47.449821: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Traceback (most recent call last):
  File "scripts_and_notebooks/lit/run.py", line 121, in <module>
    main()
  File "scripts_and_notebooks/lit/run.py", line 117, in main
    lit_demo = lit_nlp.dev_server.Server(models, datasets, port=4321)
AttributeError: module 'lit_nlp' has no attribute 'dev_server'


In [19]:
import lit_nlp
import pandas as pd
import gensim
from lit_nlp.api import dataset as lit_dataset
from lit_nlp.api import types as lit_types
from lit_nlp.api import model as lit_model
from lit_nlp import notebook
from sklearn.model_selection import train_test_split
# from lit_nlp.examples.models import model_utils
from lit_nlp.lib import utils
from typing import Iterable, Iterator
import tensorflow as tf

In [23]:
class RNNModel(lit_model.Model):
    """Wrapper for the RNN model."""

    LABELS = ['Male', 'Female']

    def __init__(self, path_to_models, **kw):
        # Load the model into memory so we're ready for interactive use.
        # FIXME: I've omitted `+ model_name + '.h5'` from the argument and which will probably break it
        self._model = tf.keras.models.load_model(path_to_models)

    # LIT API implementations
    def input_spec(self):
        """Describe the inputs to the model."""
        return {
            'dialogs': lit_types.TextSegment()
        }

    def output_spec(self):
        """Describe the model outputs."""
        return {
            # The 'parent' keyword tells LIT where to look for gold labels when computing metrics.
            'probas': lit_types.MulticlassPreds(vocab = self.LABELS, parent='label'),
        }

    # TODO: predicting as a batch should be faster
    # def predict(self, inputs: Iterable[Input]) -> Iterator[Preds]:
    # def predict(self, inputs: Iterable[tf.keras.layers.Input]):
    def predict(self, inputs):
        """Predict on a stream of examples."""
        examples = [d for d in inputs]  # any custom preprocessing (right now this does nothing)
        # returns a dict for each input
        # TODO: apply a softmax to the output
        return [{'probas': self._model.predict(example)} for example in examples]
    
    def predict_minibatch(self, inputs):
      raise ValueError("shit")


class GenderData(lit_dataset.Dataset):
    # IMPROVE ME
    """Loader of movie dialog dataset"""

    LABELS = ['Male', 'Female']

    # TODO: It should be possible to take the dataset path as an argument
    def __init__(self, path_to_data):
        # LOAD DATASET
        data = pd.read_csv(path_to_data)

        # Load word2vec trained model
        w2v = gensim.models.KeyedVectors.load("results/cornell_prepro.wordvectors", mmap='r')
        # word2vec vocabulary
        w2v_vocabulary = list(w2v.vocab.keys())

        # Keep only the words that are in the vocabulary
        # Sentences to lists of words
        docs = [d.lower().split() for d in data['text']]
        # Keep only the words that are in the w2v vocabulary (These were the words that appeared 5 times or more times)
        docs = [[word for word in doc if word in w2v_vocabulary] for doc in docs]
        # Join the list of strings into sentences
        docs = [' '.join(doc) for doc in docs]

        # Train, validation, test split
        train_split = 0.7
        val_split = 0.2
        test_split = 1 - train_split - val_split

        X = docs
        # y = np.array((data['gender'] == 'F').astype('int'))
        y = list((data['gender'] == 'F').astype('int'))

        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = test_split, random_state = 32, stratify = y)
        X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size = val_split/(train_split + val_split), random_state = 32, stratify = y_train)

        self._examples = [{
                'dialogs': x,
                'label': self.LABELS[y],
            } for x, y in zip(X_test, y_test)]

    def spec(self):
        return {
            'dialogs': lit_types.TextSegment(),
            'label': lit_types.CategoryLabel(vocab=self.LABELS)
        }

In [24]:
path_to_models = 'results/models/'

# MulitiNLIData implements the Dataset API
datasets = {
    'cornell_proprocessed_characters': GenderData('datasets/cornell_corpus/cornell_prepro_characters.csv'),
}

# NLIModel implements the Model API
models = {
    'model_rnn': RNNModel(path_to_models + 'textvectorizer_embedding128_globalmaxpooling_dense50_dense10_sigmoid' + '.h5'),
}

widget = notebook.LitWidget(models, datasets, height=800)

# lit_demo = lit_nlp.dev_server.Server(models, datasets, port=4321)
# lit_demo.serve()

ValueError: ignored

In [ ]:
widget.render()